## HO CALCOLATO I DAILY RETURN PER OGNI TITOLO ETF

In [1]:
import os
import pandas as pd

class ETFProcessor:
    def __init__(self, folder_path):
        self.folder_path = folder_path

    def daily_return_calculation(self):
        for filename in os.listdir(self.folder_path):
            if filename.endswith(".csv"):
                file_path = os.path.join(self.folder_path, filename)
                df = pd.read_csv(file_path)
                
            # Calcola la colonna 'daily_return' e moltiplica per 100
            df['daily_return'] = (df['Adj Close'].shift(0) / df['Adj Close'].shift(1) - 1) * 100

            # Salva il DataFrame aggiornato nel file CSV
            df.to_csv(file_path, index=False)

# Esempio di utilizzo
if __name__ == "__main__":
    # Sostituisci 'path_della_tua_cartella' con il percorso effettivo della tua cartella 'etfs'
    processor = ETFProcessor('C:\\Users\\stebr\\DireDSCoding\\dscoding-projects\\stefano.bruschi\\Data\\etfs')
    processor.daily_return_calculation()


## HO CALCOLATO I DAILY RETURN DEL MKT ETF

In [9]:
import os
import pandas as pd

def calculate_daily_return_mean():
    folder_path = "Data/etfs"
    output_file = os.path.join(folder_path, "mkt_values.csv")
    
    # Lista per memorizzare i DataFrame di ogni file CSV
    dfs = []

    # Leggi i file CSV nella cartella "etfs"
    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path)
            
            # Aggiungi il DataFrame alla lista
            dfs.append(df)

    # Unisci i DataFrame in base alla colonna "Date"
    result_df = pd.concat(dfs, axis=0, ignore_index=True, sort=False)

    # Calcola la media per ogni data nella colonna "daily_return"
    mean_values = result_df.groupby("Date")["daily_return"].mean().reset_index()

    # Scrivi i risultati nel nuovo file "mkt_values.csv"
    mean_values.to_csv(output_file, index=False)

if __name__ == "__main__":
    calculate_daily_return_mean()


## CALCOLO LE MEDIE MENSILI 

In [2]:
import os
import pandas as pd

# Definisci la cartella contenente i file CSV
cartella = 'Data/etfs'

# Loop attraverso tutti i file nella cartella
for file in os.listdir(cartella):
    # Verifica se il file attuale ha l'estensione ".csv"
    if file.endswith(".csv"):
        # Crea il percorso completo del file
        percorso_file = os.path.join(cartella, file)
        # Carica il file CSV in un DataFrame
        df = pd.read_csv(percorso_file)
        # Converti la colonna 'Date' in formato datetime
        df['Date'] = pd.to_datetime(df['Date'])
        # Calcola la media dei rendimenti giornalieri per ogni mese
        media_mensile = df.groupby(df['Date'].dt.to_period('M'))['daily_return'].mean()
        # Crea una nuova colonna 'monthly_mean' nel DataFrame originale e assegna i valori delle medie mensili
        df['monthly_mean'] = df['Date'].dt.to_period('M').map(media_mensile)
        # Salva il DataFrame modificato nel file CSV originale
        df.to_csv(percorso_file, index=False)


## RMSE

In [ ]:
import os
import pandas as pd
import numpy as np

# Cartella contenente i file CSV
cartella = 'Data/etfs'

# Lista dei file da elaborare (prendi tutti i file nella cartella)
elenco_file = [file for file in os.listdir(cartella) if file.endswith('.csv')]

# Loop attraverso i file specificati
for file in elenco_file:
    # Crea il percorso completo al file
    percorso_file = os.path.join(cartella, file)

    # Carica il DataFrame da ciascun file CSV
    df = pd.read_csv(percorso_file)

    # Converti la colonna 'Date' in formato datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Estrai informazioni su mese e anno dalla colonna 'Date'
    df['Mese'] = df['Date'].dt.month
    df['Anno'] = df['Date'].dt.year

    # Raggruppa i dati per mese
    gruppo_mensile = df.groupby(['Mese', 'Anno'])

    # Calcola lo scarto quadratico medio per ogni mese
    scarti_quadratici_mensili = []
    for (mese, anno), dati_mese in gruppo_mensile:
        RMSE = np.sqrt(np.mean((dati_mese['daily_return'] - dati_mese['monthly_mean']) ** 2))
        scarti_quadratici_mensili.append({
            'Mese': mese,
            'Anno': anno,
            'RMSE': RMSE
        })

    # Crea un DataFrame con gli scarti quadratici medi mensili
    df_scarti = pd.DataFrame(scarti_quadratici_mensili)

    # Unisci il DataFrame degli scarti con il DataFrame originale
    df = pd.merge(df, df_scarti, on=['Mese', 'Anno'], how='left')

    # Rimuovi le colonne 'Mese' e 'Anno'
    df = df.drop(['Mese', 'Anno'], axis=1)

    # Salva il DataFrame aggiornato nel file originale
    df.to_csv(percorso_file, index=False)
